# Predicting anomalies on NASA data using dummy primitives

This is a demo notebook that shows how to use the dummy primitives included
in the repository to build a pipeline that detects anomalies and returns them
in the `(start, stop, severity)` format.

In [1]:
import pandas as pd
from mlblocks import add_primitives_path, MLPipeline

from orion.data import load_nasa_signal

train, test = load_nasa_signal('P-3', test_size=0.3)

In [2]:
add_primitives_path('../orion/primitives')

In [13]:
pipeline = {
    'primitives': [
        'orion.estimators.MeanEstimator',
        'orion.detectors.ThresholdDetector',
        'orion.intervals.build_anomaly_intervals'
    ],
    'init_params': {
        'mlprimitives.custom.trivial.TrivialPredictor#1': {
            'method': 'mean'
        },
        'orion.detectors.ThresholdDetector#1': {
            'ratio': 0.5
        },
        'orion.intervals.build_anomaly_intervals#1': {
            'time_column': 'timestamp'
        }
    }
}

In [14]:
mlpipeline = MLPipeline.from_dict(pipeline)

In [15]:
mlpipeline.fit(train)

In [16]:
anomalies = mlpipeline.predict(test)
anomalies = pd.DataFrame(anomalies, columns=['start', 'end', 'severity'])

In [17]:
anomalies['start'] = anomalies['start'].astype(int)
anomalies['end'] = anomalies['end'].astype(int)

anomalies.head()

,start,end,severity
0,1523232000,1523253600,0.341659
1,1523275200,1523296800,0.525991
2,1523383200,1523491200,0.197364
3,1523815200,1523880000,0.069584
4,1524290400,1524312000,0.016495
